In [35]:
import numpy as np
from scipy.linalg.special_matrices import toeplitz

/var/folders/l_/vl48r50139nfhm284y3mv34r0000gn/T/ipykernel_16117/3547562558.py:2: DeprecationWarning: Please import `toeplitz` from the `scipy.linalg` namespace; the `scipy.linalg.special_matrices` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.linalg.special_matrices import toeplitz


Schoenfeld formula
$$D = \frac{(\Phi^{-1}(\beta)+\Phi^{-1}(1-\alpha))^2}{P_{cont}(1 - P_{cont}) log^2(\Delta)}$$
where 
- $D$ is the number of deaths
- $\alpha$ is the level
- $\beta$ is the power
- $P_{cont}$ is the proportion of patients in the control arm
- $\Delta$ is the hazard ratio

In [36]:
n_samples = 200
n_features_bytype = 3
n_features = 3 * n_features_bytype

In [37]:
def features_normal_cov_toeplitz(n_samples, n_features: int = 30,
                                 cov_corr: float = 0.5, dtype="float64"):
    """Normal features generator with toeplitz covariance

    An example of features obtained as samples of a centered Gaussian
    vector with a toeplitz covariance matrix

    Parameters
    ----------
    n_samples : `int`, default=200
        Number of samples

    n_features : `int`, default=30
        Number of features

    cov_corr : `float`, default=0.5
        correlation coefficient of the Toeplitz correlation matrix

    dtype : `{'float64', 'float32'}`, default='float64'
        Type of the arrays used.

    Returns
    -------
    output : `numpy.ndarray`, shape=(n_samples, n_features)
        n_samples realization of a Gaussian vector with the described
        covariance

    """
    cov = toeplitz(cov_corr ** np.arange(0, n_features))
    features = np.random.multivariate_normal(
        np.zeros(n_features), cov, size=n_samples)
    if dtype != "float64":
        return features.astype(dtype)
    return features

In [41]:
def weights_sparse_exp(n_weigths: int = 100, nnz: int = 10, scale: float = 10.,
                       dtype="float64") -> np.ndarray:
    """Sparse and exponential model weights generator

    Instance of weights for a model, given by a vector with
    exponentially decaying components: the j-th entry is given by

    .. math: (-1)^j \exp(-j / scale)

    for 0 <= j <= nnz - 1. For j >= nnz, the entry is zero.

    Parameters
    ----------
    n_weigths : `int`, default=100
        Number of weights

    nnz : `int`, default=10
        Number of non-zero weights

    scale : `float`, default=10.
        The scaling of the exponential decay

    dtype : `{'float64', 'float32'}`, default='float64'
        Type of the arrays used.

    Returns
    -------
    output : np.ndarray, shape=(n_weigths,)
        The weights vector
    """
    if nnz >= n_weigths:
        warn(("nnz must be smaller than n_weights "
              "using nnz=n_weigths instead"))
        nnz = n_weigths
    idx = np.arange(nnz)
    out = np.zeros(n_weigths, dtype=dtype)
    out[:nnz] = np.exp(-idx / scale)
    out[:nnz:2] *= -1
    return out


In [75]:
beta = np.concatenate([weights_sparse_exp(n_features_bytype,3),weights_sparse_exp(n_features_bytype,3),
                       weights_sparse_exp(n_features_bytype,3)])
treatment_effect = 0
beta = np.insert(beta, 0, treatment_effect)
print(beta.shape)

(31,)


In [76]:
X[:,10:20] = np.abs(X[:,np.arange(10,20)])
X[:,20:30] = 1 * (X[:,20:30] >= 0)

In [77]:
X = features_normal_cov_toeplitz(n_samples,n_features)
print(X.shape)

(200, 30)


In [78]:
treatment = np.random.randint(2, size=(n_samples,1))
print(treatment.shape)

(200, 1)


In [79]:
#a = np.array([1, 2, 3])
#b = np.array([4, 5, 6])
#np.vstack((a,b))
design = np.hstack((treatment,X))
marker = np.dot(design,beta)

In [109]:
a_T = 2
a_C = 1
lamb_C = 2
U = np.random.uniform(size = n_samples)
V = np.random.uniform(size = n_samples)
T = (- np.log(1-U) / np.exp(marker))**(1/a_T)
C = lamb_C * (- np.log(1-V))**(1/a_C)

In [110]:
np.mean(T <= C)

np.float64(0.655)

array([0.1424743 , 0.53856944, 0.0080225 , 0.63378701, 0.63472058,
       0.75588857, 0.16000158, 0.4029972 , 0.66234015, 0.35083583,
       0.69405028, 0.31970731, 0.38806307, 0.46906958, 0.49972919,
       0.73089186, 0.47987362, 0.27848437, 0.18742936, 0.16112311,
       0.18761813, 0.86504316, 0.70948873, 0.19031947, 0.59233024,
       0.05063032, 0.32645918, 0.15858903, 0.32506345, 0.79292716,
       0.51111554, 0.45949917, 0.02848014, 0.09177927, 0.26167691,
       0.06969013, 0.66437783, 0.29865467, 0.11957954, 0.44519762,
       0.27811318, 0.07529995, 0.27319531, 0.34711674, 0.78585914,
       0.04725044, 0.96255923, 0.14287807, 0.92352751, 0.46933116,
       0.81192721, 0.71888285, 0.48948063, 0.16896196, 0.85426913,
       0.70898226, 0.47964722, 0.92372489, 0.1829255 , 0.53164346,
       0.49567717, 0.08105117, 0.30076078, 0.71440625, 0.27221195,
       0.98439283, 0.10338828, 0.40455986, 0.91501635, 0.82194893,
       0.41782357, 0.09743789, 0.8675693 , 0.72066343, 0.01936